In [1]:
from dask.distributed import Client, LocalCluster
import dask.bag as db

In [2]:
dask_cluster = LocalCluster(n_workers=20, threads_per_worker=1)
dask_client = Client(dask_cluster)

/home/eeisenst/miniconda3/envs/cai-base/lib/python3.10/site-packages/distributed/node.py:177: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44253 instead
  warnings.warn(


In [3]:
dask_client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:44253/status,
Dashboard: http://127.0.0.1:44253/status,Workers: 20
Total threads: 20,Total memory: 31.17 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38129,Workers: 20
Dashboard: http://127.0.0.1:44253/status,Total threads: 20
Started: Just now,Total memory: 31.17 GiB
Comm: tcp://127.0.0.1:36767,Total threads: 1
Dashboard: http://127.0.0.1:40157/status,Memory: 1.56 GiB
Nanny: tcp://127.0.0.1:45225,


# Prepare the pairs dataset

In [4]:
import re
import glob

from bs4 import BeautifulSoup, SoupStrainer

from tqdm.notebook import tqdm

In [5]:
# test_fn = glob.glob("../../tibert_data/84000/data-translation-memory/*.tmx")[0]

In [6]:
# with open(test_fn, mode='r', encoding='utf-8') as f:
#     soup = BeautifulSoup(
#         f,
#         'xml',
#         from_encoding='UTF8')

In [7]:
def _repl_split_commas_or_kill(match, take_second_item=False, test_mode=False):
    # Take a regex match of the form (...,...), or some other brackets, and return one of the items.
    #
    # Test mode returns the bracketed input and the result, for unit testing.
    match = match.group(0)
    if ',' in match:
        if take_second_item:
            if test_mode:
                return "@{}|{}@".format(match, match.split(',')[1][:-1])
            else:
                return match.split(',')[1][:-1]
        return match.split(',')[0][1:]
    # This is a different comma in unicode but it looks the same in most fonts
    if '‚' in match:
        if take_second_item:
            if test_mode:
                return "@{}|{}@".format(match, match.split('‚')[1][:-1])
            else:
                return match.split('‚')[1][:-1]
        return match.split('‚')[0][1:]
    return ''


def _strip_out_pairs(soup):
    strip_chars = {'!'}
    bad_chars = {'_', 'a', 'b', 'c', 'd', 'e', 'f', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y'}
    pairs = []
    for sent_pair_tag in soup.find_all("tu"):
        cur_pair = [' '.join(list(tag.stripped_strings)) for tag in sent_pair_tag.find_all("tuv")]
        assert len(cur_pair) == 2
        tibetan, english = cur_pair
        if len(set(tibetan).intersection(bad_chars)) > 0:
            continue
        for c in strip_chars:
            tibetan = tibetan.replace(c, '')
        tibetan = re.sub(
            r"[({][^(){}]*?[})]",
            _repl_split_commas_or_kill,
            tibetan)
        pairs.append((tibetan, english))
    return pairs

In [8]:
def _process(fn, with_locator=False):
    with open(fn, mode='r', encoding='utf-8') as f:
        soup = BeautifulSoup(
            f,
            'xml',
            from_encoding='UTF8')
    cur_pairs = _strip_out_pairs(soup)
    if with_locator:
        return zip([fn] * len(cur_pairs), cur_pairs)
    else:
        return cur_pairs

In [9]:
# pairs = _process("../../tibert_data/84000/data-translation-memory\Toh_134-The_Absorption_That_Encapsulates_All_Merit-v2.tmx")

In [120]:
pairs = []
for fn in tqdm(glob.glob("../../tibert_data/84000/data-translation-memory/*.tmx")):
    cur_pairs = _process(fn)
    pairs.extend(zip([fn] * len(cur_pairs), cur_pairs))

In [10]:
pairs = db \
    .from_sequence(glob.glob("../../tibert_data/84000/data-translation-memory/*.tmx")) \
    .map(lambda x: _process(x, with_locator=True)) \
    .flatten() \
    .compute()

In [11]:
len(pairs)

92145

In [12]:
pairs[-1]

('../../tibert_data/84000/data-translation-memory\\Toh_99-The_Blessed_One%27s_Wisdom_That_Leads_to_Infinite_Certainty-v2.tmx',
 ('རྒྱ་གར་གྱི་མཁན་པོ་བི་ཤུད་དྷ་སིང་ཧ་དང་། སརྦ་ཛྙཱ་དེ་བ་དང་། ཞུ་ཆེན་གྱི་ལོཙྪ་བ་བན་དེ་དཔལ་བརྩེགས་ཀྱིས་ཞུས་ཏེ་གཏན་ལ་ཕབ་པ།། །།',
  'The text was later edited and finalized by the Indian preceptors Śuddhasiṃha and Sarvajñādeva, together with the translator-editor, venerable Paltsek.'))

# Find bad characters and unusual patterns

In [13]:
# set([x for x, _ in pairs]), set([y for _, y in pairs])

In [13]:
tibetan_chars, english_chars = set(), set()

bad_chars = {'_', 'a', 'b', 'c', 'd', 'e', 'f', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y'}

cnt = 0
for fn, (tibetan, english) in tqdm(pairs):
    assert len(set(tibetan).intersection(bad_chars)) == 0
    if '—' in english:
        print(fn, tibetan, english)
        cnt += 1
        if cnt > 10:
            break
    tibetan_chars = tibetan_chars.union(set(tibetan))
    english_chars = english_chars.union(set(english))
cnt, len(pairs)
# tibetan_chars, english_chars

../../tibert_data/84000/data-translation-memory\Toh_100-The_Ornament_of_the_Light_of_Awareness-v1.tmx ཕྱོགས་བཅུའི་འཇིག་རྟེན་གྱི་ཁམས་སྟོང་གསུམ་གྱི་སྟོང་ཆེན་པོའི་འཇིག་རྟེན་གྱི་ཁམས་གྲངས་མེད་ཅིང་བསམ་གྱིས་མི་ཁྱབ་པ་དག་གི་རྡུལ་ཕྲ་རབ་ཀྱི་རྡུལ་སྙེད་དག་འོད་ཟེར་ཆེན་པོའི་སྤྲིན་དག་གིས་སྣང་བར་མཛད་དོ། the Bhagavat illuminated the ten directions with great clouds of light rays that shone in as many worlds as there are atoms of dust in the great trichiliocosm—an uncountable and inconceivable number.
../../tibert_data/84000/data-translation-memory\Toh_100-The_Ornament_of_the_Light_of_Awareness-v1.tmx འཇམ་དཔལ་དེའི་ཚེ་ན་ཆོས་ཀྱི་རྔ་བོ་ཆེ་དེ་བལྟར་ “At that time, Mañjuśrī, due to the sound of that great Dharma drum—
../../tibert_data/84000/data-translation-memory\Toh_100-The_Ornament_of_the_Light_of_Awareness-v1.tmx དེ་ལྟ་བུ་དེའི་སྒྲ་དེས་སུམ་ཅུ་རྩ་གསུམ་པའི་རིས་ཀྱི་ལྷ་ཐམས་ཅད་ལ་ཡང་དག་པར་བསྐུལ་བས་དེ་དག་འཇིགས་པ་དང་། སྐྲག་པ་དང་། སྐྱོ་བ་དང་། ཡིད་བྱུང་བར་གྱུར་ནས་ —all the gods in the heaven of the Thirty-Three are 

(11, 92145)

In [15]:
pairs[-500]

('../../tibert_data/84000/data-translation-memory\\Toh_99-The_Blessed_One%27s_Wisdom_That_Leads_to_Infinite_Certainty-v2.tmx',
 ('མཽད་གལ་གྱི་བུ་དེ་ཇི་སྙམ་དུ་སེམས། དེའི་ཚེ་ དེའི་དུས་ན། དྲང་སྲོང་སྲིད་པ་ཞི་བྱེད་ཅེས་བྱ་བར་གྱུར་པ་དེ་གཞན་ཡིན་པར་སེམས་ན། ཁྱོད་ཀྱིས་དེ་ལྟར་མི་བལྟ་སྟེ།',
  '“Maudgalyāyana, in case you may think that at that time the sage Pacifier of Existence was anyone else, do not think so.'))

In [16]:
import unicodedata

In [ ]:
sorted([(c, unicodedata.name(c)) for c in tibetan_chars])

In [ ]:
sorted([(c, unicodedata.name(c)) for c in english_chars])

# Test how well the English side tokenizes

In [14]:
from transformers import AutoTokenizer

In [28]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [29]:
test_text = pairs[-500][1][1]

In [30]:
test_text = "then you should know that they will be restricted to two levels—the level of the śrāvakas and the level of the pratyekabuddhas."

In [31]:
print(test_text)
ids, tokens = tokenizer.encode(test_text), tokenizer.tokenize(test_text)
' '.join([str(id_) + '|' + token for id_, token in zip(ids, tokens)])

then you should know that they will be restricted to two levels—the level of the śrāvakas and the level of the pratyekabuddhas.


'101|then 1173|you 1128|should 1431|know 1221|that 1115|they 1152|will 1209|be 1129|restricted 7458|to 1106|two 1160|levels 3001|— 783|the 1103|level 1634|of 1104|the 1103|ś 319|##r 1197|##ā 9663|##va 2497|##kas 13257|and 1105|the 1103|level 1634|of 1104|the 1103|p 185|##rat 7625|##ye 4980|##ka 1968|##bu 7925|##dd 13976|##has 16481|.'

# Test the loader objects

In [4]:
from dask.distributed import Client, LocalCluster
import dask.bag as db

In [5]:
dask_cluster = LocalCluster(n_workers=20, threads_per_worker=1)
dask_client = Client(dask_cluster)

/home/eeisenst/miniconda3/envs/cai-base/lib/python3.10/site-packages/distributed/node.py:177: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36055 instead
  warnings.warn(


In [6]:
dask_client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:36055/status,
Dashboard: http://127.0.0.1:36055/status,Workers: 20
Total threads: 20,Total memory: 31.17 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41149,Workers: 20
Dashboard: http://127.0.0.1:36055/status,Total threads: 20
Started: Just now,Total memory: 31.17 GiB
Comm: tcp://127.0.0.1:35853,Total threads: 1
Dashboard: http://127.0.0.1:44655/status,Memory: 1.56 GiB
Nanny: tcp://127.0.0.1:43525,


In [8]:
from cai_common.data import ParallelTXMLoader

In [9]:
txm_loader = ParallelTXMLoader()

In [10]:
txm_bag = txm_loader \
    .apply_markup(apply=True) \
    .clean_bad_chars(clean=True) \
    .to_bag()

In [11]:
txm_bag.count().compute()

98345

In [12]:
txm_bag.take(10)

(('86',
  'F.116.b',
  '1',
  'སངས་རྒྱས་དང་བྱང་ཆུབ་སེམས་དཔའ་ཐམས་ཅད་ལ་ཕྱག་འཚལ་ལོ། །',
  'Homage to all the buddhas and bodhisattvas!'),
 ('86',
  'F.116.b',
  '45',
  'འདི་སྐད་བདག་གིས་ཐོས་པའི་དུས་གཅིག་ན།',
  'Thus have I heard at one time.'),
 ('86',
  'F.116.b',
  '206',
  'དེ་ནས་བཅོམ་ལྡན་འདས་འཁོར་བརྒྱ་སྟོང་དུ་མས་ཡོངས་སུ་བསྐོར་ཅིང་མདུན་གྱིས་བལྟས་ནས་ཆོས་སྟོན་ཏོ། །',
  'The Blessed One, surrounded by a retinue of many hundreds of thousands, surveyed those before him, and taught the Dharma.'),
 ('86',
  'F.116.b',
  '328',
  'དེའི་ཚེ་བྱང་ཆུབ་སེམས་དཔའ་སེམས་དཔའ་ཆེན་པོ་བྱམས་པ་ཞེས་བྱ་བ་འཁོར་དེ་ཉིད་དུ་འདུས་པར་གྱུར་ཏེ་འདུག་གོ',
  'At that time, the bodhisattva mahāsattva Maitreya was present among that very retinue.'),
 ('86',
  'F.116.b',
  '415',
  'དེ་ནས་བྱང་ཆུབ་སེམས་དཔའ་སེམས་དཔའ་ཆེན་པོ་བྱམས་པ་སྟན་ལས་ལངས་ཏེ་',
  'He rose from his seat,'),
 ('86',
  'F.116.b',
  '444',
  'བླ་གོས་ཕྲག་པ་གཅིག་ཏུ་གཟར་ནས་པུས་མོ་གཡས་པའི་ལྷ་ང་ས་ལ་བཙུགས་ཏེ།',
  'draped his shawl over one shoulder, set his right knee 

In [15]:
txm_bag \
    .map(lambda pair: pair[3]) \
    .flatten() \
    .distinct() \
    .count() \
    .compute()

86

In [17]:
all_letters = txm_bag \
    .map(lambda pair: pair[3]) \
    .flatten() \
    .distinct() \
    .compute()
' '.join(sorted(all_letters))

'\n   ༄ ༅ ་ ། ༑ ཀ ཁ ག ང ཅ ཆ ཇ ཉ ཊ ཋ ཌ ཎ ཏ ཐ ད ན པ ཕ བ མ ཙ ཚ ཛ ཝ ཞ ཟ འ ཡ ར ལ ཤ ཥ ས ཧ ཨ ཪ ཱ ི ུ ཱུ ེ ཻ ོ ཽ ཾ ཿ ྀ ྃ ྅ ྐ ྑ ྒ ྔ ྕ ྗ ྙ ྚ ྛ ྜ ྞ ྟ ྠ ྡ ྣ ྤ ྥ ྦ ྨ ྩ ྪ ྫ ྭ ྱ ྲ ླ ྴ ྵ ྶ ྷ'

In [18]:
txm_df = txm_loader.dataframe

In [19]:
txm_df.head(20)

,filename,tohoku,folio,position,tibetan,english
0,Toh_86-The_Question_of_Maitreya_on_the_Eight_Q...,86,F.116.b,1,སངས་རྒྱས་དང་བྱང་ཆུབ་སེམས་དཔའ་ཐམས་ཅད་ལ་ཕྱག་འཚལ་...,Homage to all the buddhas and bodhisattvas!
1,Toh_86-The_Question_of_Maitreya_on_the_Eight_Q...,86,F.116.b,45,འདི་སྐད་བདག་གིས་ཐོས་པའི་དུས་གཅིག་ན།,Thus have I heard at one time.
2,Toh_86-The_Question_of_Maitreya_on_the_Eight_Q...,86,F.116.b,206,དེ་ནས་བཅོམ་ལྡན་འདས་འཁོར་བརྒྱ་སྟོང་དུ་མས་ཡོངས་ས...,"The Blessed One, surrounded by a retinue of ma..."
3,Toh_86-The_Question_of_Maitreya_on_the_Eight_Q...,86,F.116.b,328,དེའི་ཚེ་བྱང་ཆུབ་སེམས་དཔའ་སེམས་དཔའ་ཆེན་པོ་བྱམས་...,"At that time, the bodhisattva mahāsattva Maitr..."
4,Toh_86-The_Question_of_Maitreya_on_the_Eight_Q...,86,F.116.b,415,དེ་ནས་བྱང་ཆུབ་སེམས་དཔའ་སེམས་དཔའ་ཆེན་པོ་བྱམས་པ་...,"He rose from his seat,"
5,Toh_86-The_Question_of_Maitreya_on_the_Eight_Q...,86,F.116.b,444,བླ་གོས་ཕྲག་པ་གཅིག་ཏུ་གཟར་ནས་པུས་མོ་གཡས་པའི་ལྷ་...,"draped his shawl over one shoulder, set his ri..."
6,Toh_86-The_Question_of_Maitreya_on_the_Eight_Q...,86,F.116.b,514,བཅོམ་ལྡན་འདས་ག་ལ་བ་དེར་ལོགས་སུ་ཐལ་མོ་སྦྱར་བ་བཏ...,and paid homage with palms joined toward the B...
7,Toh_86-The_Question_of_Maitreya_on_the_Eight_Q...,86,F.117.b,222,ཁྱད་པར་ཐོབ་པར་འགྱུར།,will gain special attainments;
8,Toh_86-The_Question_of_Maitreya_on_the_Eight_Q...,86,F.117.b,253,བདུད་ཕྱིར་རྒོལ་བ་བཅོམ་པར་འགྱུར།,they will vanquish negative and hostile influe...
9,Toh_86-The_Question_of_Maitreya_on_the_Eight_Q...,86,F.117.b,358,ཆོས་ཐམས་ཅད་ཀྱི་ངོ་བོ་ཉིད་ཀྱི་མཚན་ཉིད་ཇི་ལྟ་བ་བ...,they will realize the intrinsic characteristic...


# Test the Dataset object

In [1]:
from tibert.models import TibertTokenizer
from tibert.models.parallel_sents_dataset import ParallelSentencesDataset

In [2]:
from transformers import AlbertModel, BartModel, AutoConfig, AutoModel, AutoTokenizer

In [3]:
tibert_tkn = TibertTokenizer.from_pretrained('tibert-bpe-large')
tibert_tkn.stochastic_tokenization = False

In [4]:
bart_tkn = AutoTokenizer.from_pretrained('facebook/bart-base')

In [5]:
bart_tkn("this test this", max_length=128, padding="max_length", truncation=True)

{'input_ids': [0, 9226, 1296, 42, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [6]:
dataset = ParallelSentencesDataset(tibert_tkn, bart_tkn)

In [7]:
# dataset[0]

In [8]:
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.sampler import RandomSampler

In [14]:
data_loader = DataLoader(
    dataset,
    batch_size=8,
    sampler=RandomSampler(dataset),
    collate_fn=dataset.collate_fn,
    drop_last=False)
data_iter = iter(data_loader)

In [10]:
# next(iter(data_loader))

## Test metrics calculations

In [11]:
from tibert.models import TiEBARTForConditionalGeneration

In [27]:
res_dict = TiEBARTForConditionalGeneration.from_base_models("c:/workspace/tibert_data/training/models/tibert-champion/946000.bin")
tiebart_model, tibert_tkn, tibert_model, bart_model, bart_tkn = \
    [res_dict[key] for key in ["tiebart_model", "tibert_tokenizer", "tibert_model", "bart_model", "bart_tokenizer"]]

In [13]:
tiebart_model.config.encoder.vocab_size, tiebart_model.config.decoder.vocab_size

(10000, 50265)

In [15]:
batch = next(data_iter)

In [24]:
cross_entropy, _, _ = tiebart_model(**batch)
float(cross_entropy)

13.959240913391113

In [19]:
generated_ids = tiebart_model.generate(
    batch['input_ids'],
    attention_mask=batch["attention_mask"],
    use_cache=True,
    decoder_start_token_id=tiebart_model.config.decoder.bos_token_id)
generated_ids.shape

torch.Size([8, 20])

In [21]:
batch['labels'].shape

torch.Size([8, 39])

In [25]:
def ids_to_clean_text(tokenizer, generated_ids):
    gen_text = tokenizer.batch_decode(
        generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    return list(map(str.strip, gen_text))

In [29]:
preds = ids_to_clean_text(bart_tkn, generated_ids)
target = ids_to_clean_text(bart_tkn, batch["labels"])

In [39]:
sum(list(map(len, preds))) / len(preds)

36.75

In [31]:
preds, target

(['""""((((((((""""""',
  '""""""""""""\'\'\'\'\'\'',
  ", think' think' think' think' think''''''''",
  'or or or////////////////',
  '..................... kids kids kids kids kids kids kids kids kids kids kids',
  '.... ( ( ( ( (////////////',
  '" \'\' (\' do\'; good; good; good;............',
  '"" (" \'[ \'[ \'[ \'[ \'[ \'[ \'[ \'[ \'[ \'[ \'[ \'[ \'[ \'['],
 ['When Māra, the evil one, witnessed the Bodhisattva’s power and the displays that he performed, his mind was disturbed with jealousy and avarice.',
  '“ Kāśyapa, I have not proclaimed that suffering is created by itself,” the Blessed One replied.',
  'If one plays dice, one will win.',
  'please shower the rain of Dharma!',
  'Then came twenty thousand horses covered with golden ornaments and full of jewels.',
  '“The young woman saw that the young brahmin Sumati was well proportioned and beautiful, and she became quite attached to him and fell in love.',
  'He is the leader of the herd like a perfect bull, always patient a

# Test processing modular tags

In [11]:
import glob

from bs4 import BeautifulSoup

from tqdm.notebook import tqdm

In [20]:
for fn in tqdm(glob.glob("../../tibert_data/84000/data-translation-memory/*.tmx")):
    with open(fn, mode='r', encoding='utf-8') as f:
        soup = BeautifulSoup(f.read(), 'xml')
        bad_ct = 0
        for sent_pair_tag in soup.find_all("tu"):
            bad_ct += len([tag for tag in sent_pair_tag.find_all("tuv") if tag.get('xml:lang') not in ['bo', 'en']])
        if bad_ct > 0:
            print(fn, bad_ct)
#         print([tag for tag in tqdm(sent_pair_tag.find_all("tuv"), leave=False) if tag.get('xml:lang') not in ['bo', 'en']])

../../tibert_data/84000/data-translation-memory\Modular_Passages-TM-Test.tmx 5



# Compare the aligned and unaligned datasets

In [1]:
from dask.distributed import Client, LocalCluster
import dask.bag as db

In [2]:
dask_cluster = LocalCluster(n_workers=20, threads_per_worker=1)
dask_client = Client(dask_cluster)

In [3]:
dask_client

Client Scheduler: tcp://127.0.0.1:57890 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 20 Cores: 20 Memory: 17.10 GB


In [4]:
from tibert.data import ParallelTXMLoader

In [5]:
txm_loader = ParallelTXMLoader('../../tibert_data/')

In [6]:
txm_df = txm_loader \
    .apply_markup(apply=True) \
    .clean_bad_chars(clean=True) \
    .dataframe

In [7]:
len(txm_df)

Task exception was never retrieved
future: <Task finished coro=<connect.<locals>._() done, defined at C:\tools\miniconda3\envs\tibert\lib\site-packages\distributed\comm\core.py:279> exception=CommClosedError()>
Traceback (most recent call last):
  File "C:\tools\miniconda3\envs\tibert\lib\site-packages\distributed\comm\core.py", line 288, in _
    handshake = await asyncio.wait_for(comm.read(), 1)
  File "C:\tools\miniconda3\envs\tibert\lib\asyncio\tasks.py", line 435, in wait_for
    await waiter
concurrent.futures._base.CancelledError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\tools\miniconda3\envs\tibert\lib\site-packages\distributed\comm\core.py", line 295, in _
    raise CommClosedError()
distributed.comm.core.CommClosedError


98345

In [8]:
txm_df.head(10)

,filename,tohoku,tibetan,english
0,Modular_Passages-TM-Test.tmx,Passages,རྒྱལ་པོ་ཞེས་བྱ་བས་རྒྱལ་སྲིད་འབྱོར་པ། རྒྱས་པ་བད...,"Under his rule, the kingdom prospered and thri..."
1,Modular_Passages-TM-Test.tmx,Passages,དེས་དཔུང་གི་ཚོགས་ཡན་ལག་བཞི་པ་གླང་པོ་ཆེ་པའི་ཚོག...,he called up the four branches of his armed fo...
2,Modular_Passages-TM-Test.tmx,Passages,སུམ་ཅུ་རྩ་གསུམ་པའི་ལྷ་རྣམས་ཀྱི་ཁ་དོག་གི་མཐུ་བས...,bathed in a vast light more luminous than the ...
3,Modular_Passages-TM-Test.tmx,Passages,མ་མ་བརྒྱད་པོ་པང་ན་འཚོ་བའི་མ་མ་གཉིས་དང། ནུ་མ་སྣ...,was entrusted to eight nursemaids: two to cudd...
4,Modular_Passages-TM-Test.tmx,Passages,རྒྱལ་པོ་རྒྱལ་རིགས་སྤྱི་བོར་དབང་བསྐུར་བ། ལྗོངས་...,He trained in and mastered those arts and skil...
5,Modular_Passages-TM-Test.tmx,Passages,དེ་ཡང་རྒྱལ་པོ་རྒྱལ་རིགས་སྤྱི་བོར་དབང་བསྐུར་བ་ཡ...,He too has been crowned and anointed a kṣatriy...
6,Modular_Passages-TM-Test.tmx,Passages,ལག་པ་འགྲམ་པ་ལ་བསྟད་ནས་སེམས་ཁོང་དུ་ཆུད་ཅིང་འདུག...,"Head in hands, he sat and sat, absorbed in tho..."
7,Modular_Passages-TM-Test.tmx,Passages,ཡུལ་དང་སྲོག་ལ་གནོད་གྱུར་ན། །སྐྱེས་བུས་ཀུན་ཏུ་ས...,"When land and life are threatened, Seek always..."
8,Modular_Passages-TM-Test.tmx,Passages,དེས་རིགས་མཉམ་པ་ལས་ཆུང་མ་བླངས་ནས། དེ་དེ་དང་ལྷན་...,took a wife of equal caste and together they s...
9,Modular_Passages-TM-Test.tmx,Passages,ཀློག་པའི་ཕ་རོལ་ཏུ་སོན་ཅིང་། དེའི་ཚེ་ན་བྲམ་ཟེའི...,"Once he had mastered letters, he was instructe..."


In [9]:
chars = txm_df.tibetan.map(lambda x: list(x)).explode().unique().compute().tolist()
sorted(chars)

['\n',
 ' ',
 '༄',
 '༅',
 '་',
 '།',
 '༑',
 'ཀ',
 'ཁ',
 'ག',
 'ང',
 'ཅ',
 'ཆ',
 'ཇ',
 'ཉ',
 'ཊ',
 'ཋ',
 'ཌ',
 'ཎ',
 'ཏ',
 'ཐ',
 'ད',
 'ན',
 'པ',
 'ཕ',
 'བ',
 'མ',
 'ཙ',
 'ཚ',
 'ཛ',
 'ཝ',
 'ཞ',
 'ཟ',
 'འ',
 'ཡ',
 'ར',
 'ལ',
 'ཤ',
 'ཥ',
 'ས',
 'ཧ',
 'ཨ',
 'ཪ',
 'ཱ',
 'ི',
 'ུ',
 'ཱུ',
 'ེ',
 'ཻ',
 'ོ',
 'ཽ',
 'ཾ',
 'ཿ',
 'ྀ',
 'ྃ',
 '྅',
 'ྐ',
 'ྑ',
 'ྒ',
 'ྔ',
 'ྕ',
 'ྗ',
 'ྙ',
 'ྚ',
 'ྛ',
 'ྜ',
 'ྞ',
 'ྟ',
 'ྠ',
 'ྡ',
 'ྣ',
 'ྤ',
 'ྥ',
 'ྦ',
 'ྨ',
 'ྩ',
 'ྪ',
 'ྫ',
 'ྭ',
 'ྱ',
 'ྲ',
 'ླ',
 'ྴ',
 'ྵ',
 'ྶ',
 'ྷ']

In [10]:
len(txm_df.groupby('tohoku').english.apply(lambda s: s.str.cat().split('.')).explode())

C:\tools\miniconda3\envs\tibert\lib\site-packages\ipykernel_launcher.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  """Entry point for launching an IPython kernel.


65377

In [11]:
from tibert.data import TeiLoader

In [12]:
parallel_df = dask_client.persist(TeiLoader("../../tibert_data/", "kangyur").dataframe)

In [13]:
len(parallel_df.groupby('tohoku_number').text.apply(lambda s: s.str.cat().split('.')).explode())

C:\tools\miniconda3\envs\tibert\lib\site-packages\ipykernel_launcher.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  """Entry point for launching an IPython kernel.


166183

# Fix bad tags

In [17]:
from bs4 import BeautifulSoup, SoupStrainer

In [18]:
with open("../../tibert_data/84000/data-tei/translations/kangyur/translations/039-002_toh46-the_teaching_on_clearing_the_boundless_gates.xml", mode='r', encoding='utf-8') as f:
    soup = BeautifulSoup(f.read(), 'xml', parse_only=SoupStrainer("div", type="translation"))

In [20]:
for ref in soup.find_all("canonDef"):
    print(ref)

<canonDef ref="UT22084-039-002-354">
<distinct>dhāraṇī</distinct> is a
                            term that refers to knowledge of the connections between phenomena.
                            Blessed One, <distinct>dhāraṇī</distinct> means to maintain the act of
                            recollection. They are called <distinct>dhāraṇī</distinct> because they
                            teach the bases of secret words that maintain the power of the treasure
                            of insight. <ref cRef="F.55.b" type="folio"/>. In this way, I will
                            uphold the infinite power of awakening with a mind that utilizes
                            expertise in wisdom.</canonDef>
<canonDef ref="UT22084-039-002-360">Anantavyūha, the thus-gone ones have four types of fearlessness.
                            Pratyekabuddhas do not have them, why speak of śrāvakas and other
                            worldly beings? What are these four types of fearlessness?
    

In [18]:
with open("../../tibert_data/84000/data-tei/translations/kangyur/translations/096-062_toh813,1098-a_prayer_from_destroyer_of_the_great_chiliocosm_cf_toh_558.xml", mode='r', encoding='utf-8') as f:
    soup = BeautifulSoup(f.read(), 'xml')

In [21]:
refs_to_strip = {'bampo', 'sanskrit', 'volume'}

toh_key, num_toh_keys = 'toh813', 2

for ref in soup.find_all("ref"):
    print(ref)
#     if ref.attrs.get('type', None) in refs_to_strip:
#         ref.decompose()
#     else:
#         if 'key' not in ref.attrs and num_toh_keys > 1:
#             raise Exception("No 'key' attribute in ref tag {}, but more than one Tohoku number for the text {}"
#                             .format(ref, toh_key))
#         if not ('key' not in ref.attrs or ref.attrs.get('key', None) == toh_key) or 'cRef' not in ref.attrs:
#             ref.decompose()
# for div in soup.find_all("div"):
#     div.unwrap()

<ref>Toh 813</ref>
<ref>Toh 1098</ref>
<ref target="https://read.84000.co/translation/toh558.html#UT22084-090-002-771">1.250</ref>
<ref target="https://read.84000.co/translation/toh558.html#UT22084-090-002-274">1.265</ref>
<ref target="https://read.84000.co/translation/toh558.html#UT22084-090-002-319">1.310</ref>
<ref target="https://read.84000.co/translation/toh558.html#UT22084-090-002-323">1.314</ref>
<ref target="https://read.84000.co/translation/toh558.html#UT22084-090-002-14">1.4</ref>
<ref target="https://read.84000.co/translation/toh558.html#UT22084-090-002-253">1.243</ref>
<ref target="https://read.84000.co/translation/toh558.html#UT22084-090-002-259">1.249</ref>
<ref target="https://archive.org/details/sacredbooksofbud16londuoft/page/208/mode/2up">p. 208 
                                                          et seq.</ref>
<ref target="https://archive.org/details/sacredbooksofbud16londuoft/page/242/mode/2up">p. 242 et
                                                  seq</r

In [32]:
import glob
import re

from tqdm.notebook import tqdm

from bs4 import BeautifulSoup, SoupStrainer

In [27]:
tag_treatments = {
    'term': 'unwrap',
    'lg': 'unwrap',
    'lb': 'decompose',
    'milestone': 'decompose',
    'foreign': 'unwrap',
    'distinct': 'unwrap',
    'emph': 'unwrap',
    'title': 'unwrap',
    'head': 'unwrap',
    'q': 'unwrap',
    'l': 'unwrap',
    'trailer': 'unwrap',
    'note': 'decompose',
    'p': 'unwrap',
    'mantra': 'unwrap',
    'list': 'unwrap',
    'item': 'unwrap',
    'label': 'decompose'}

In [28]:
def _treat_tags(soup, tag_treatments):
    # Apply the tag_treatments dictionary to the XML soup. Return the soup so that it can be fed to other functions.
    for tag_name, tag_treatment in tag_treatments.items():
        for elem in soup.find_all(tag_name):
            if tag_treatment == 'decompose':
                elem.decompose()
            elif tag_treatment == 'unwrap':
                elem.unwrap()
    return soup

In [41]:
for fn in tqdm(glob.glob("../../tibert_data/84000/data-tei/translations/kangyur/translations/*.xml")):
    bt_count = 0
    with open(fn, mode='r', encoding='utf-8') as f:
        toh_re_res = re.compile(r"_toh[\d,]+[-_]").search(fn)
        if toh_re_res:
            toh_keys = ['toh' + toh_num for toh_num in toh_re_res.group(0)[4:-1].split(',')]
            num_toh_keys = len(toh_keys)
        if num_toh_keys == 1:
            continue
        fstr = f.read()
        soup = BeautifulSoup(fstr, 'xml', parse_only=SoupStrainer("div", type="translation"))
        soup = _treat_tags(soup, tag_treatments)
        for ref in soup.find_all("ref"):
            if 'key' not in ref.attrs and num_toh_keys > 1:
                bt_count += 1
    if bt_count > 0:
        print(fn, bt_count)

../../tibert_data/84000/data-tei/translations/kangyur/translations\057-004_toh145,847-the_dharani_of_the_jewel_torch.xml 2
../../tibert_data/84000/data-tei/translations/kangyur/translations\096-062_toh813,1098-aspiration_prayer_from_destroyer_of_the_great_trichiliocosm_cf_toh_558.xml 1
../../tibert_data/84000/data-tei/translations/kangyur/translations\096-062_toh813,1098-a_prayer_from_destroyer_of_the_great_chiliocosm_cf_toh_558.xml 1



In [53]:
fn = "../../tibert_data/84000/data-tei/translations/kangyur/translations/096-062_toh813,1098-a_prayer_from_destroyer_of_the_great_chiliocosm_cf_toh_558.xml"
with open(fn, mode='r', encoding='utf-8') as f:
    fstr = f.read()
    soup = BeautifulSoup(fstr, 'xml', parse_only=SoupStrainer("div", type="translation"))
    toh_re_res = re.compile(r"_toh[\d,]+[-_]").search(fn)
    if toh_re_res:
        toh_keys = ['toh' + toh_num for toh_num in toh_re_res.group(0)[4:-1].split(',')]
        num_toh_keys = len(toh_keys)
    soup = _treat_tags(soup, tag_treatments)
    for ref in soup.find_all("ref"):
        if 'key' not in ref.attrs and num_toh_keys > 1:
            print(ref)

<ref target="https://read.84000-translate.org/translation/toh558.html#UT22084-090-002-459">bhūta</ref>
